In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input/titanic"))

['train.csv', 'test.csv', 'gender_submission.csv']


In [2]:
train=pd.read_csv("../input/titanic/train.csv")
test=pd.read_csv("../input/titanic/test.csv")
submission=pd.read_csv("../input/titanic/gender_submission.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [6]:
train.groupby(by=['Survived','Pclass']).count()['PassengerId']

Survived  Pclass
0         1          80
          2          97
          3         372
1         1         136
          2          87
          3         119
Name: PassengerId, dtype: int64

In [7]:
train.groupby(by=['Survived','Sex','Pclass']).count()['PassengerId']

Survived  Sex     Pclass
0         female  1           3
                  2           6
                  3          72
          male    1          77
                  2          91
                  3         300
1         female  1          91
                  2          70
                  3          72
          male    1          45
                  2          17
                  3          47
Name: PassengerId, dtype: int64

In [8]:
def make_cat(x):
    if x=='male':
        return 1
    else:
        return 0
train['sex_cat']=train['Sex'].apply(lambda x: make_cat(x))    
test['sex_cat']=test['Sex'].apply(lambda x: make_cat(x))    

In [9]:
print(train.Embarked.unique())
print(test.Embarked.unique())

['S' 'C' 'Q' nan]
['Q' 'S' 'C']


In [10]:
print(train.Embarked.value_counts())
print(test.Embarked.value_counts())

S    644
C    168
Q     77
Name: Embarked, dtype: int64
S    270
C    102
Q     46
Name: Embarked, dtype: int64


In [11]:
train[train.Embarked.isnull()].index

Int64Index([61, 829], dtype='int64')

In [12]:
train.at[61,'Embarked']='S'
train.at[829,'Embarked']='S'

In [13]:
train.describe(include='O').T

,count,unique,top,freq
Name,891,891,"Braund, Mr. Owen Harris",1
Sex,891,2,male,577
Ticket,891,681,347082,7
Cabin,204,147,B96 B98,4
Embarked,891,3,S,646


In [14]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292
sex_cat,891.0,0.647587,0.477990,0.00,0.0000,1.0000,1.0,1.0000


In [15]:
train.drop(['PassengerId','Sex','Name','Ticket','Cabin'],axis=1,inplace=True)
test.drop(['PassengerId','Sex','Name','Ticket','Cabin'],axis=1,inplace=True)

In [16]:
train['Age'].fillna(train['Age'].mode()[0], inplace=True)
test['Age'].fillna(test['Age'].mode()[0], inplace=True)
test['Fare'].fillna(test['Fare'].mode()[0], inplace=True)


In [17]:
train.describe(include='O').T

,count,unique,top,freq
Embarked,891,3,S,646


In [18]:
test.describe(include='O').T

,count,unique,top,freq
Embarked,418,3,S,270


In [19]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,891.0,28.566970,13.199572,0.42,22.0000,24.0000,35.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292
sex_cat,891.0,0.647587,0.477990,0.00,0.0000,1.0000,1.0,1.0000


In [20]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
Pclass,418.0,2.265550,0.841838,1.00,1.0000,3.0000,3.000000,3.0000
Age,418.0,28.364833,13.180116,0.17,21.0000,24.0000,35.750000,76.0000
SibSp,418.0,0.447368,0.896760,0.00,0.0000,0.0000,1.000000,8.0000
Parch,418.0,0.392344,0.981429,0.00,0.0000,0.0000,0.000000,9.0000
Fare,418.0,35.560497,55.857145,0.00,7.8958,14.4542,31.471875,512.3292
sex_cat,418.0,0.636364,0.481622,0.00,0.0000,1.0000,1.000000,1.0000


In [21]:
def make_embark_cat(x):
    if x=='S':
        return 0
    elif x=='C':
        return 1
    else:
        return 2
train['Embarked_cat']=train['Embarked'].apply(lambda x: make_embark_cat(x))    
train.drop('Embarked',axis=1,inplace=True)
test['Embarked_cat']=test['Embarked'].apply(lambda x: make_embark_cat(x))    
test.drop('Embarked',axis=1,inplace=True)
    

In [22]:
y=train['Survived']
X=train.drop(['Survived'],axis=1)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)
classifier = DecisionTreeClassifier(random_state=0)
classifier = classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred))

Accuracy Score: 0.7597765363128491


In [24]:
from sklearn.ensemble import RandomForestClassifier
clf_2 = RandomForestClassifier(max_depth=10, random_state=0)
clf_2.fit(X_train, y_train)
y_pred = clf_2.predict(X_test)
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred))

Accuracy Score: 0.8435754189944135


In [25]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred))

Accuracy Score: 0.776536312849162


In [26]:
from sklearn import svm
sc_X = StandardScaler()
clf_3 = svm.SVC()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
clf_3.fit(X_train, y_train)
y_pred = clf_3.predict(X_test)
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred))

Accuracy Score: 0.8212290502793296


In [27]:
test_predict=clf_2.predict(test)

In [28]:
submission['Survived']=test_predict

In [29]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [30]:
submission.to_csv('submission.csv', index=False)